In [1]:
from typing import List
import pandas as pd

In [2]:
attrib_test_file = r'C:\Users\S0053071\Repos\Orion_Process_Backup\HdgRpts_Results\202502\Attrib\Orion_Asset_Attrib_Results_TEST.xlsx'
sheet_name = 'AssetAttrib_Detailed'
dt_flds = ['Attrib_StartDt', 'Attrib_EndDt', 'HedgeDt','Seg_StartDt','TradeDt','ExpiryDt']

all_summary_cols = [
    'Ntnl_BoP',
    'Ntnl_Added',
    'Ntnl_Matured',
    'Ntnl_EoP',
    'MV_BoP',
    'MV_Chg_New',
    'MV_Chg_Ntnl_Added',
    'MV_Chg_Ntnl_Chg',
    'MV_Chg_Ntnl_Decr',
    'MV_Chg_Ntnl_Matured',
    'MV_Chg_PayOff',
    'MV_Chg_Spot',
    'MV_Chg_RFR',
    'MV_Chg_Dvd',
    'MV_Chg_Vol',
    'MV_Chg_Time',
    'MV_EoP'
]

In [3]:
def convert_to_date(cell_value):
    """Converts a cell value to date, handling NaT."""
    if pd.isna(cell_value):
        return pd.NaT
    return pd.to_datetime(cell_value).date()

def read_excel_df_with_dates(xl_file_path: str, sheet_name: str, date_flds: List[str]):

    # Create a date converter for pandas to read date cols in excel as dates
    # date_flds = ['HedgeDt']
    converters = {col: convert_to_date for col in date_flds}

    xl_df = pd.read_excel(xl_file_path, sheet_name=sheet_name, converters=converters)

    return xl_df

In [4]:
detailed_results = read_excel_df_with_dates(attrib_test_file, sheet_name, dt_flds)
columns_to_sum = [col for col in all_summary_cols if col in detailed_results.columns]

In [7]:
def sum_specific_columns(detailed_results: pd.DataFrame, columns_to_sum: List[str]) -> pd.DataFrame:
    """
    Sums specific columns of the detailed_results DataFrame and returns a new DataFrame with the results.
    """
    # Sum the specified columns
    summed_data = detailed_results[columns_to_sum].sum()

    # Convert the summed data to a DataFrame
    summed_df = pd.DataFrame(summed_data)
    # summed_df = pd.DataFrame(summed_data).transpose()

    return summed_df



In [8]:
summed_df = sum_specific_columns(detailed_results, columns_to_sum)
print(summed_df)

                                0
Ntnl_BoP             1.817466e+10
Ntnl_Added           1.512000e+08
Ntnl_Matured        -1.298236e+08
Ntnl_EoP             1.819603e+10
MV_BoP               1.268710e+09
MV_Chg_New           7.378199e+06
MV_Chg_Ntnl_Added   -1.338689e+05
MV_Chg_Ntnl_Chg      0.000000e+00
MV_Chg_Ntnl_Decr     0.000000e+00
MV_Chg_Ntnl_Matured -8.702231e+04
MV_Chg_PayOff       -1.332862e+07
MV_Chg_Spot          7.656599e+06
MV_Chg_RFR           2.006838e+04
MV_Chg_Dvd          -2.216253e+05
MV_Chg_Vol           3.505209e+05
MV_Chg_Time         -7.376911e+05
MV_EoP               1.269606e+09


In [39]:
def get_attrib_results_agg_summary(detailed_results: pd.DataFrame, sum_cols: List[str], position_type: str):

    # Calculate Results In Aggregate
    attrib_min_dt = detailed_results['Attrib_StartDt'].min()
    attrib_max_dt = detailed_results['Attrib_EndDt'].max()

    ntnl_bop = detailed_results[detailed_results['Attrib_StartDt']==attrib_min_dt]['Ntnl_BoP'].sum()
    mv_bop = detailed_results[detailed_results['Attrib_StartDt']==attrib_min_dt]['MV_BoP'].sum()

    ntnl_eop = detailed_results[detailed_results['Attrib_EndDt']==attrib_max_dt]['Ntnl_EoP'].sum()
    mv_eop = detailed_results[detailed_results['Attrib_EndDt']==attrib_max_dt]['MV_EoP'].sum()

    agg_sum_cols = [col for col in sum_cols if col not in ['Ntnl_BoP', 'MV_BoP', 'Ntnl_EoP', 'MV_EoP']]

    attrib_totals = pd.DataFrame(detailed_results[agg_sum_cols].sum()).transpose()
    attrib_totals['Ntnl_BoP'] = ntnl_bop
    attrib_totals['MV_BoP'] = mv_bop
    attrib_totals['Ntnl_EoP'] = ntnl_eop
    attrib_totals['MV_EoP'] = mv_eop
    attrib_totals['PositionType'] = position_type
    attrib_totals['Attrib_StartDt'] = detailed_results['Attrib_StartDt'].min()
    attrib_totals['Attrib_EndDt'] = detailed_results['Attrib_EndDt'].max()

    ordered_cols = ['PositionType', 'Attrib_StartDt', 'Attrib_EndDt'] + sum_cols
    # ordered_cols = ['Attrib_StartDt', 'Attrib_EndDt'] + sum_cols
    attrib_totals = attrib_totals[ordered_cols]
    # ordered_cols = sum_cols
    # attrib_totals = attrib_totals[ordered_cols]

    # Temporarily Convert to Series
    # temp_series = attrib_totals.iloc[0].squeeze()
    # attrib_totals = temp_series.to_frame(name=position_type)


    return attrib_totals

In [40]:
new_summary = get_attrib_results_agg_summary(detailed_results, columns_to_sum, 'OrionAsset')

In [41]:
new_summary

,PositionType,Attrib_StartDt,Attrib_EndDt,Ntnl_BoP,Ntnl_Added,Ntnl_Matured,Ntnl_EoP,MV_BoP,MV_Chg_New,MV_Chg_Ntnl_Added,MV_Chg_Ntnl_Chg,MV_Chg_Ntnl_Decr,MV_Chg_Ntnl_Matured,MV_Chg_PayOff,MV_Chg_Spot,MV_Chg_RFR,MV_Chg_Dvd,MV_Chg_Vol,MV_Chg_Time,MV_EoP
0,OrionAsset,2024-12-31,2025-02-05,7.820954e+08,151200000.0,-1.298236e+08,8.034718e+08,5.512420e+07,7378198.621,-133868.921009,0.0,0.0,-87022.313177,-1.332862e+07,7.656599e+06,20068.382821,-221625.334447,350520.854227,-737691.082451,5.602076e+07


In [33]:
new_summary

,OrionAsset
Attrib_StartDt,2024-12-31
Attrib_EndDt,2025-02-05
Ntnl_BoP,782095419.117612
Ntnl_Added,151200000.0
Ntnl_Matured,-129823569.401294
Ntnl_EoP,803471849.716318
MV_BoP,55124198.891422
MV_Chg_New,7378198.621
MV_Chg_Ntnl_Added,-133868.921009
MV_Chg_Ntnl_Chg,0.0


In [23]:
new_summary.transpose()

,0
Attrib_StartDt,2024-12-31
Attrib_EndDt,2025-02-05
PositionType,OrionAsset
Ntnl_BoP,782095419.117612
Ntnl_Added,151200000.0
Ntnl_Matured,-129823569.401294
Ntnl_EoP,803471849.716318
MV_BoP,55124198.891422
MV_Chg_New,7378198.621
MV_Chg_Ntnl_Added,-133868.921009


In [15]:
type(new_summary)

pandas.core.frame.DataFrame

In [28]:
detailed_results

,Attrib_StartDt,Attrib_EndDt,HedgeDt,Seg_StartDt,TradeDt,ExpiryDt,CompID,Bbg_Idx,Fund_Name,Opt_Type,...,MV_Chg_Ntnl_Decr,MV_Chg_Ntnl_Matured,MV_Chg_PayOff,MV_Chg_Spot,MV_Chg_RFR,MV_Chg_Dvd,MV_Chg_Vol,MV_Chg_Time,MV_EoP,Check
0,2024-12-31,2025-01-02,2024-01-01,2023-12-29,2024-01-02,2025-01-02,1,NDX Index,NASDAQ-100 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,...,0,61.018421,-2.531989e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0
1,2024-12-31,2025-01-02,2024-01-01,2023-12-29,2024-01-02,2025-01-02,1,SPMARC5P Index,S&P MARC 5% 1 Yr Pt-to-Pt Uncapped,Call,...,0,5471.227344,-1.708117e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0
2,2024-12-31,2025-01-02,2024-01-01,2023-12-29,2024-01-02,2025-01-02,1,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,...,0,971.754499,-4.032341e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0
3,2024-12-31,2025-01-02,2024-01-01,2023-12-29,2024-01-02,2025-01-02,1,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap & Floor,Call Spread,...,0,41.957682,-1.741054e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0
4,2024-12-31,2025-01-02,2024-01-01,2023-12-29,2024-01-02,2025-01-02,1,SPX Index,S&P 500 1 Yr Pt-to-Pt Multiplier w Cap,Call Spread,...,0,140.978160,-5.849941e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1683,2025-02-04,2025-02-05,2025-02-01,2025-01-31,2025-02-03,2026-01-30,1,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap,Call Spread,...,0,0.000000,0.000000e+00,51689.931315,-1434.858544,-3514.342062,295.672824,-1882.577496,2.709569e+06,0.0
1684,2025-02-04,2025-02-05,2025-02-01,2025-01-31,2025-02-03,2026-01-30,1,SPX Index,S&P 500 1 Yr Pt-to-Pt 100 Part w Cap & Floor,Call Spread,...,0,0.000000,0.000000e+00,2128.326529,-59.190808,-144.676811,73.015966,-77.330844,1.107076e+05,0.0
1685,2025-02-04,2025-02-05,2025-02-01,2025-01-31,2025-02-03,2026-01-30,1,SPX Index,S&P 500 1 Yr Pt-to-Pt Multiplier w Cap,Call Spread,...,0,0.000000,0.000000e+00,7130.248981,-202.393605,-485.439936,874.653475,-280.353634,3.446555e+05,0.0
1686,2025-02-04,2025-02-05,2025-02-01,2025-01-31,2025-02-03,2026-01-30,1,SPX Index,S&P 500 1 Yr Pt-to-Pt Uncapped w Fee,Call,...,0,0.000000,0.000000e+00,16695.902599,-539.439685,-1152.421457,-5197.439138,-1136.233167,3.856790e+05,0.0


In [35]:
new_summary.to_clipboard(index=True)